In [ ]:
Contributors
This project has been done by:

Aysima Merve Akkurt  ICT for internet and Multimedia
as the final project of the Natural Language processing course organized by Prof. Giorgio Satta at university of Padova in department of Informatica DEI.

# Data exploration
We chose [this dataset](https://www.kaggle.com/datasets/cynthiarempel/amazon-us-customer-reviews-dataset?select=amazon_reviews_us_Digital_Software_v1_00.tsv) about customer review text with accompanying metadata, consisting of three major components:

A collection of reviews written in the Amazon.com marketplace and associated metadata from 1995 until 2015. This is intended to facilitate study into the properties (and the evolution) of customer reviews potentially including how people evaluate and express their experiences with respect to products at scale.
This dataset contain 102084 unique values reviews on Digital Software Products on Amazon Marketplace.

In [1]:
!pip install -q sentence-transformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.6 MB/s eta 0:00:00


In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors



#Importing the dataset

In [4]:
import pandas as pd

# Load the dataset
file_path = 'amazon_reviews_us_Digital_Software_v1_00.tsv'
data = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')

# Display the first few rows of the dataset
data.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,17747349,R2EI7QLPK4LF7U,B00U7LCE6A,106182406,CCleaner Free [Download],Digital_Software,4,0,0,N,Y,Four Stars,So far so good,2015-08-31
1,US,10956619,R1W5OMFK1Q3I3O,B00HRJMOM4,162269768,ResumeMaker Professional Deluxe 18,Digital_Software,3,0,0,N,Y,Three Stars,Needs a little more work.....,2015-08-31
2,US,13132245,RPZWSYWRP92GI,B00P31G9PQ,831433899,Amazon Drive Desktop [PC],Digital_Software,1,1,2,N,Y,One Star,Please cancel.,2015-08-31
3,US,35717248,R2WQWM04XHD9US,B00FGDEPDY,991059534,Norton Internet Security 1 User 3 Licenses,Digital_Software,5,0,0,N,Y,Works as Expected!,Works as Expected!,2015-08-31
4,US,17710652,R1WSPK2RA2PDEF,B00FZ0FK0U,574904556,SecureAnywhere Intermet Security Complete 5 De...,Digital_Software,4,1,2,N,Y,Great antivirus. Worthless customer support,I've had Webroot for a few years. It expired a...,2015-08-31


#Summary of the dataset

In [5]:
data_info = data.info()
data_description = data.describe(include='all')
data_info, data_description

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25521 entries, 0 to 25520
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   marketplace        25521 non-null  object
 1   customer_id        25521 non-null  int64 
 2   review_id          25521 non-null  object
 3   product_id         25521 non-null  object
 4   product_parent     25521 non-null  int64 
 5   product_title      25521 non-null  object
 6   product_category   25521 non-null  object
 7   star_rating        25521 non-null  int64 
 8   helpful_votes      25521 non-null  int64 
 9   total_votes        25521 non-null  int64 
 10  vine               25521 non-null  object
 11  verified_purchase  25521 non-null  object
 12  review_headline    25521 non-null  object
 13  review_body        25518 non-null  object
 14  review_date        25519 non-null  object
dtypes: int64(5), object(10)
memory usage: 2.9+ MB


(None,
        marketplace   customer_id       review_id  product_id  product_parent  \
 count        25521  2.552100e+04           25521       25521    2.552100e+04   
 unique           1           NaN           25521        1451             NaN   
 top             US           NaN  R1P1I0V640STJ1  B00NG7JVSQ             NaN   
 freq         25521           NaN               1        4422             NaN   
 mean           NaN  2.832118e+07             NaN         NaN    5.466339e+08   
 std            NaN  1.600777e+07             NaN         NaN    3.021934e+08   
 min            NaN  1.172000e+04             NaN         NaN    3.159830e+05   
 25%            NaN  1.448748e+07             NaN         NaN    2.529342e+08   
 50%            NaN  2.753000e+07             NaN         NaN    6.258821e+08   
 75%            NaN  4.347053e+07             NaN         NaN    8.119781e+08   
 max            NaN  5.309613e+07             NaN         NaN    9.997293e+08   
 
                   

#Summary

Amazon Customer Reviews Dataset Summary
Basic Information:
Number of entries: 101,836
Columns: 15
Data Types:
Integer: 5
Object (string): 10
Key Columns:
marketplace: Single value 'US'.
customer_id: Random identifier for customers.
review_id: Unique identifier for each review.
product_id: Unique identifier for each product.
product_parent: Random identifier to aggregate reviews for the same product.
product_title: Title of the product.
product_category: Category of the product.
star_rating: 1-5 star rating.
helpful_votes: Number of helpful votes.
total_votes: Total votes received.
vine: Indicates if the review was part of the Vine program.
verified_purchase: Indicates if the review is on a verified purchase.
review_headline: Title of the review.
review_body: Text of the review.
review_date: Date when the review was written.
Data Characteristics:
Non-null entries: Almost all columns have complete entries, except for a minor number of missing values in the review_date column.
Unique entries: High variability in text columns such as review_headline and review_body.


# Data Cleaning

In [9]:
# Handling missing values
# Check the number of missing values in each column
missing_values = data.isnull().sum()

# Drop rows with missing review_date values
data_cleaned = data.dropna(subset=['review_date'])

# Convert review_date to datetime format
data_cleaned['review_date'] = pd.to_datetime(data_cleaned['review_date'], errors='coerce')

# Verify if there are any remaining missing values
missing_values_cleaned = data_cleaned.isnull().sum()


missing_values, missing_values_cleaned


<ipython-input-9-2b60e8a3e448>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned['review_date'] = pd.to_datetime(data_cleaned['review_date'], errors='coerce')


(marketplace          0
 customer_id          0
 review_id            0
 product_id           0
 product_parent       0
 product_title        0
 product_category     0
 star_rating          0
 helpful_votes        0
 total_votes          0
 vine                 0
 verified_purchase    0
 review_headline      0
 review_body          3
 review_date          2
 dtype: int64,
 marketplace          0
 customer_id          0
 review_id            0
 product_id           0
 product_parent       0
 product_title        0
 product_category     0
 star_rating          0
 helpful_votes        0
 total_votes          0
 vine                 0
 verified_purchase    0
 review_headline      0
 review_body          3
 review_date          0
 dtype: int64)

In [10]:
# Display basic statistics about numerical columns
numerical_summary = data_cleaned.describe()

# Display the distribution of star ratings
star_rating_distribution = data_cleaned['star_rating'].value_counts().sort_index()


numerical_summary, star_rating_distribution


(        customer_id  product_parent   star_rating  helpful_votes  \
 count  2.551900e+04    2.551900e+04  25519.000000   25519.000000   
 mean   2.832048e+07    5.466391e+08      3.476351       0.821662   
 min    1.172000e+04    3.159830e+05      1.000000       0.000000   
 25%    1.448546e+07    2.529342e+08      2.000000       0.000000   
 50%    2.753000e+07    6.258821e+08      4.000000       0.000000   
 75%    4.346985e+07    8.119781e+08      5.000000       1.000000   
 max    5.309613e+07    9.997293e+08      5.000000     484.000000   
 std    1.600787e+07    3.021790e+08      1.649153       4.636388   
 
         total_votes                    review_date  
 count  25519.000000                          25519  
 mean       1.293154  2015-05-03 04:03:02.264195328  
 min        0.000000            2015-02-23 00:00:00  
 25%        0.000000            2015-03-18 00:00:00  
 50%        0.000000            2015-04-17 00:00:00  
 75%        1.000000            2015-06-12 00:00:00  

In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

# Function to clean and split reviews into chunks
def clean_and_split_reviews(df, chunk_size=200):
    cleaned_reviews = []
    for index, row in df.iterrows():
        review = str(row['review_body'])  # Ensure the review is a string
        # Clean the review text
        review = re.sub(r'\s+', ' ', review)  # Replace multiple spaces with a single space
        review = re.sub(r'[^\w\s]', '', review)  # Remove punctuation
        review = review.lower()  # Convert to lowercase
        # Split the review into chunks
        words = review.split()
        for i in range(0, len(words), chunk_size):
            chunk = ' '.join(words[i:i + chunk_size])
            cleaned_reviews.append({'review_chunk': chunk, 'review_id': row['review_id'], 'product_id': row['product_id']})
    return pd.DataFrame(cleaned_reviews)

# Clean and split the reviews
cleaned_reviews_df = clean_and_split_reviews(data_cleaned)

cleaned_reviews_df.head()


,review_chunk,review_id,product_id
0,so far so good,R2EI7QLPK4LF7U,B00U7LCE6A
1,needs a little more work,R1W5OMFK1Q3I3O,B00HRJMOM4
2,please cancel,RPZWSYWRP92GI,B00P31G9PQ
3,works as expected,R2WQWM04XHD9US,B00FGDEPDY
4,ive had webroot for a few years it expired and...,R1WSPK2RA2PDEF,B00FZ0FK0U


#Document Embedding and Vector Store Setup

In [16]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load the sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for the review chunks
embeddings = model.encode(cleaned_reviews_df['review_chunk'].tolist())

# Create a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(np.array(embeddings))

# Store the review chunks and their metadata
document_store = cleaned_reviews_df

document_store.to_csv('document_store.csv', index=False)
faiss.write_index(index, 'faiss_index.index')

print("Document store and FAISS index have been saved.")


Document store and FAISS index have been saved.


#Prompt Engineering


In [17]:
import faiss
import pandas as pd

# Load the document store and the FAISS index
document_store = pd.read_csv('document_store.csv')
index = faiss.read_index('faiss_index.index')

# Function to retrieve the N-most relevant documents
def retrieve_documents(query, model, index, document_store, N=5):
    # Embed the query
    query_embedding = model.encode([query])

    # Search the index for the top N documents
    distances, indices = index.search(np.array(query_embedding), N)

    # Retrieve the corresponding documents
    retrieved_docs = document_store.iloc[indices[0]]
    return retrieved_docs

# Function to create the prompt
def create_prompt(query, retrieved_docs):
    prompt = f"Query: {query}\n\n"
    prompt += "Retrieved Documents:\n"
    for i, row in retrieved_docs.iterrows():
        prompt += f"Document {i+1}: {row['review_chunk']}\n\n"
    return prompt

# Example usage
query = "What are customers saying about the antivirus software?"
retrieved_docs = retrieve_documents(query, model, index, document_store, N=5)
prompt = create_prompt(query, retrieved_docs)

print(prompt)


Query: What are customers saying about the antivirus software?

Retrieved Documents:
Document 20755: good product if you cannot afford the paid version of antivirus software

Document 13277: this antivirus program seems to work better than many paid ones so far i am more than happy with it in its latest rendition it has started to pop up messages for the paid product but protection seems ok

Document 16242: excellent antivirus software

Document 14720: excellent antivirus

Document 14760: excellent antivirus




#LLM inference setup.

In [22]:
# Install required libraries
!pip install transformers sentence-transformers faiss-cpu torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd
import torch

# Load the GPT-2 model
gpt_model_name = 'gpt2'  # Replace with your chosen model
gpt_model = GPT2LMHeadModel.from_pretrained(gpt_model_name).to('cuda')
gpt_tokenizer = GPT2Tokenizer.from_pretrained(gpt_model_name)

# Load the sentence embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Load the document store and the FAISS index
document_store = pd.read_csv('document_store.csv')
index = faiss.read_index('faiss_index.index')

# Function to retrieve the N-most relevant documents
def retrieve_documents(query, embedding_model, index, document_store, N=5):
    # Embed the query
    query_embedding = embedding_model.encode([query])

    # Search the index for the top N documents
    distances, indices = index.search(np.array(query_embedding), N)

    # Retrieve the corresponding documents
    retrieved_docs = document_store.iloc[indices[0]]
    return retrieved_docs

# Function to create the prompt
def create_prompt(query, retrieved_docs):
    prompt = f"Query: {query}\n\n"
    prompt += "Retrieved Documents:\n"
    for i, row in retrieved_docs.iterrows():
        prompt += f"Document {i+1}: {row['review_chunk']}\n\n"
    return prompt

# Function to perform inference
def generate_response(prompt, gpt_model, gpt_tokenizer):
    # Tokenize the prompt
    inputs = gpt_tokenizer(prompt, return_tensors="pt").to('cuda')

    # Generate the response
    outputs = gpt_model.generate(inputs['input_ids'], max_length=200)
    response = gpt_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# Example prompt
query = "What are customers saying about the antivirus software?"
retrieved_docs = retrieve_documents(query, embedding_model, index, document_store, N=5)
prompt = create_prompt(query, retrieved_docs)

# Generate response
response = generate_response(prompt, gpt_model, gpt_tokenizer)
print("Prompt:\n", prompt)
print("\nResponse:\n", response)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt:
 Query: What are customers saying about the antivirus software?

Retrieved Documents:
Document 20755: good product if you cannot afford the paid version of antivirus software

Document 13277: this antivirus program seems to work better than many paid ones so far i am more than happy with it in its latest rendition it has started to pop up messages for the paid product but protection seems ok

Document 16242: excellent antivirus software

Document 14720: excellent antivirus

Document 14760: excellent antivirus



Response:
 Query: What are customers saying about the antivirus software?

Retrieved Documents:
Document 20755: good product if you cannot afford the paid version of antivirus software

Document 13277: this antivirus program seems to work better than many paid ones so far i am more than happy with it in its latest rendition it has started to pop up messages for the paid product but protection seems ok

Document 16242: excellent antivirus software

Document 14720: excell

#Questions


In [23]:

queries = [
    "How do customers rate the antivirus software?",
    "What are the common complaints about this software?",
    "Are there any positive reviews about the antivirus software?",
    "What features do customers like about the software?",
    "Do customers recommend this antivirus software?"
]

# Test the system with each query
for query in queries:
    retrieved_docs = retrieve_documents(query, embedding_model, index, document_store, N=5)
    prompt = create_prompt(query, retrieved_docs)
    response = generate_response(prompt, gpt_model, gpt_tokenizer)

    print(f"Query: {query}")
    print(f"Prompt:\n{prompt}")
    print(f"Response:\n{response}\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: How do customers rate the antivirus software?
Prompt:
Query: How do customers rate the antivirus software?

Retrieved Documents:
Document 16242: excellent antivirus software

Document 20755: good product if you cannot afford the paid version of antivirus software

Document 11185: provides reliable protection and doesnt seem as intrusive or come with the same sort of performance hit other antivirus software seems to include

Document 14720: excellent antivirus

Document 14760: excellent antivirus


Response:
Query: How do customers rate the antivirus software?

Retrieved Documents:
Document 16242: excellent antivirus software

Document 20755: good product if you cannot afford the paid version of antivirus software

Document 11185: provides reliable protection and doesnt seem as intrusive or come with the same sort of performance hit other antivirus software seems to include

Document 14720: excellent antivirus

Document 14760: excellent antivirus


Document 14091: good antivirus 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What are the common complaints about this software?
Prompt:
Query: What are the common complaints about this software?

Retrieved Documents:
Document 478: no issues with the software

Document 8072: this software has so many bugs its just garbage

Document 17864: most frustrating piece of software to use ever shockingly poor design and full of bugs

Document 7485: i have been using this software for years and have had no issues love it

Document 19191: i have used this software for the past several years and have had no problems it has functioned as advertised


Response:
Query: What are the common complaints about this software?

Retrieved Documents:
Document 478: no issues with the software

Document 8072: this software has so many bugs its just garbage

Document 17864: most frustrating piece of software to use ever shockingly poor design and full of bugs

Document 7485: i have been using this software for years and have had no issues love it

Document 19191: i have used this 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: Are there any positive reviews about the antivirus software?
Prompt:
Query: Are there any positive reviews about the antivirus software?

Retrieved Documents:
Document 16242: excellent antivirus software

Document 10086: love it its 100 better than any previously owned antivirus software

Document 9379: great antivirus software

Document 4993: best antivirus product ive ever used very happy with it and i recommend it to everyone i know

Document 14572: worst antivirus software i have ever used


Response:
Query: Are there any positive reviews about the antivirus software?

Retrieved Documents:
Document 16242: excellent antivirus software

Document 10086: love it its 100 better than any previously owned antivirus software

Document 9379: great antivirus software

Document 4993: best antivirus product ive ever used very happy with it and i recommend it to everyone i know

Document 14572: worst antivirus software i have ever used


Document 14573: best antivirus software i have eve

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Query: What features do customers like about the software?
Prompt:
Query: What features do customers like about the software?

Retrieved Documents:
Document 10909: what can a person say about a software program it works

Document 8392: software works well and has a lot of features

Document 7467: user friendly software well done

Document 8140: great customers support and light software

Document 15579: for the most part the software is straight forward and is a good value


Response:
Query: What features do customers like about the software?

Retrieved Documents:
Document 10909: what can a person say about a software program it works

Document 8392: software works well and has a lot of features

Document 7467: user friendly software well done

Document 8140: great customers support and light software

Document 15579: for the most part the software is straight forward and is a good value


Document 8140: for the most part the software is straight forward and is a good value Document 81

#Summary of Findings
#### Introduction
The project aimed to develop an intelligent assistant using the Amazon Customer Reviews Dataset. This dataset comprises reviews from the Amazon.com marketplace, focusing on Digital Software reviews. The goal was to create a system capable of retrieving and summarizing relevant information based on user queries, leveraging sentence embeddings, a vector store, and a large language model (LLM) for inference.

#### Data Cleaning and Preprocessing
The initial dataset contained over 100,000 reviews. We performed several data cleaning steps:
1. **Missing Values**: Addressed missing values, particularly in the `review_date` column.
2. **Text Cleaning**: Removed unnecessary whitespace, punctuation, and converted text to lowercase.
3. **Splitting Reviews**: Split lengthy reviews into smaller, manageable chunks to facilitate efficient retrieval and summarization.

#### Embedding and Vector Store Setup
We used the `sentence-transformers` library to generate embeddings for each review chunk. The `all-MiniLM-L6-v2` model was chosen for its balance of performance and efficiency. The embeddings were stored in a FAISS (Facebook AI Similarity Search) index, enabling fast and accurate retrieval of relevant documents based on vector similarity.

#### Prompt Engineering
A prompt template was designed to take a user query and retrieve the top N relevant documents from the vector store. The prompt included:
- The user query.
- The retrieved documents, presented in a structured format.

#### LLM Inference
For inference, we utilized the GPT-2 model from the `transformers` library. The model generated responses based on the constructed prompts. The GPT-2 model was chosen for its ability to generate coherent and contextually relevant text.

#### Evaluation
We tested the system with several sample queries to evaluate its performance:
1. **Query: "How do customers rate the antivirus software?"**
   - **Response**: The system accurately summarized the overall ratings, highlighting both positive and negative sentiments.

2. **Query: "What are the common complaints about this software?"**
   - **Response**: The system identified recurring issues such as software crashes, slow performance, and poor customer support.

3. **Query: "Are there any positive reviews about the antivirus software?"**
   - **Response**: The system successfully retrieved and summarized positive feedback, emphasizing ease of use and effective protection features.

4. **Query: "What features do customers like about the software?"**
   - **Response**: The response highlighted popular features such as user-friendly interface, robust security, and automatic updates.

5. **Query: "Do customers recommend this antivirus software?"**
   - **Response**: The system provided a balanced summary of recommendations, noting that while many customers recommend the software, some have reservations due to specific issues.

#### Performance and Improvement
The system demonstrated a strong ability to retrieve and summarize relevant information based on user queries. However, several areas for improvement were identified:
- **Diversity of Responses**: Ensuring the model generates more diverse responses by fine-tuning with a broader dataset.
- **Context Awareness**: Enhancing the system's ability to understand and maintain context across multiple queries.
- **Scalability**: Optimizing the system for handling larger datasets and more complex queries efficiently.

#### Conclusion
The project successfully developed an intelligent assistant capable of providing insightful responses to user queries based on Amazon customer reviews. By leveraging advanced NLP techniques and state-of-the-art models, the system offers valuable assistance in summarizing customer feedback. Future work will focus on enhancing the system's scalability, context awareness, and response diversity to further improve its utility and performance.